In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
                    .appName('Read and Write') \
                    .config('spark.jars.packages', 'mysql:mysql-connector-java:8.0.26') \
                    .getOrCreate()

spark

2021-09-29 02:38:45,165 WARN util.Utils: Your hostname, localhost resolves to a loopback address: 127.0.0.1; using 192.168.96.2 instead (on interface eth0)
2021-09-29 02:38:45,166 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4bf91da4-8a47-4ba8-912e-9ee76cf5b6bd;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/spark-3.1.2-bin-without-hadoop/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found mysql#mysql-connector-java;8.0.26 in central
	found com.google.protobuf#protobuf-java;3.11.4 in central
:: resolution report :: resolve 92ms :: artifacts dl 2ms
	:: modules in use:
	com.google.protobuf#protobuf-java;3.11.4 from central in [default]
	mysql#mysql-connector-java;8.0.26 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-4bf91da4-8a47-4ba8-912e-9ee76cf5b6bd
	confs: [default]
	0 artifacts copied, 2 already retrieved (0kB/3ms)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For Sp

## CSV

In [4]:
csv_df = spark.read \
              .options(delimeter=',') \
              .csv('datasets/forestfires.csv', header=True, inferSchema=True)
csv_df.show()
csv_df.count()

+---+---+-----+---+----+-----+-----+----+----+---+----+----+----+
|  X|  Y|month|day|FFMC|  DMC|   DC| ISI|temp| RH|wind|rain|area|
+---+---+-----+---+----+-----+-----+----+----+---+----+----+----+
|  7|  5|  mar|fri|86.2| 26.2| 94.3| 5.1| 8.2| 51| 6.7| 0.0| 0.0|
|  7|  4|  oct|tue|90.6| 35.4|669.1| 6.7|18.0| 33| 0.9| 0.0| 0.0|
|  7|  4|  oct|sat|90.6| 43.7|686.9| 6.7|14.6| 33| 1.3| 0.0| 0.0|
|  8|  6|  mar|fri|91.7| 33.3| 77.5| 9.0| 8.3| 97| 4.0| 0.2| 0.0|
|  8|  6|  mar|sun|89.3| 51.3|102.2| 9.6|11.4| 99| 1.8| 0.0| 0.0|
|  8|  6|  aug|sun|92.3| 85.3|488.0|14.7|22.2| 29| 5.4| 0.0| 0.0|
|  8|  6|  aug|mon|92.3| 88.9|495.6| 8.5|24.1| 27| 3.1| 0.0| 0.0|
|  8|  6|  aug|mon|91.5|145.4|608.2|10.7| 8.0| 86| 2.2| 0.0| 0.0|
|  8|  6|  sep|tue|91.0|129.5|692.6| 7.0|13.1| 63| 5.4| 0.0| 0.0|
|  7|  5|  sep|sat|92.5| 88.0|698.6| 7.1|22.8| 40| 4.0| 0.0| 0.0|
|  7|  5|  sep|sat|92.5| 88.0|698.6| 7.1|17.8| 51| 7.2| 0.0| 0.0|
|  7|  5|  sep|sat|92.8| 73.2|713.0|22.6|19.3| 38| 4.0| 0.0| 0.0|
|  6|  5| 

517

In [5]:
csv_df.write.csv('output/write-csv', mode='overwrite')

## Parquet

In [2]:
parquet_df = spark.read \
                  .parquet('datasets/olympic-history/athletes_events')
parquet_df.show()
parquet_df.count()

+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+--------------------+--------------------+-----+
| ID|                Name|Sex|Age|Height|Weight|          Team|NOC|      Games|Year|Season|       City|               Sport|               Event|Medal|
+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+--------------------+--------------------+-----+
|  1|           A Dijiang|  M| 24|   180|    80|         China|CHN|1992 Summer|1992|Summer|  Barcelona|          Basketball|Basketball Men's ...|   NA|
|  2|            A Lamusi|  M| 23|   170|    60|         China|CHN|2012 Summer|2012|Summer|     London|                Judo|Judo Men's Extra-...|   NA|
|  3| Gunnar Nielsen Aaby|  M| 24|    NA|    NA|       Denmark|DEN|1920 Summer|1920|Summer|  Antwerpen|            Football|Football Men's Fo...|   NA|
|  4|Edgar Lindenau Aabye|  M| 34|    NA|    NA|Denmark/Sweden|DEN|1900 Summer|1900|Summ

271116

In [7]:
parquet_df.write.parquet('output/write-parquet', mode='overwrite')

## Database

In [12]:
# Setup the JDBC connection
jdbc_url = "jdbc:mysql://mysql:3306/sakila"
connection_properties = {
      "user" : "root",
      "password" : "root",
      "driver" : "com.mysql.jdbc.Driver"
    }

# Create a query
query = """
            SELECT * FROM actor
        """

# run the query
bh_bairros_df = spark.read \
                     .jdbc(url=jdbc_url, 
                           table=f"({query}) AS t", 
                           properties=connection_properties)

bh_bairros_df.show()
bh_bairros_df.count()
bh_bairros_df.printSchema()

+--------+----------+------------+-------------------+
|actor_id|first_name|   last_name|        last_update|
+--------+----------+------------+-------------------+
|       1|  PENELOPE|     GUINESS|2006-02-15 04:34:33|
|       2|      NICK|    WAHLBERG|2006-02-15 04:34:33|
|       3|        ED|       CHASE|2006-02-15 04:34:33|
|       4|  JENNIFER|       DAVIS|2006-02-15 04:34:33|
|       5|    JOHNNY|LOLLOBRIGIDA|2006-02-15 04:34:33|
|       6|     BETTE|   NICHOLSON|2006-02-15 04:34:33|
|       7|     GRACE|      MOSTEL|2006-02-15 04:34:33|
|       8|   MATTHEW|   JOHANSSON|2006-02-15 04:34:33|
|       9|       JOE|       SWANK|2006-02-15 04:34:33|
|      10| CHRISTIAN|       GABLE|2006-02-15 04:34:33|
|      11|      ZERO|        CAGE|2006-02-15 04:34:33|
|      12|      KARL|       BERRY|2006-02-15 04:34:33|
|      13|       UMA|        WOOD|2006-02-15 04:34:33|
|      14|    VIVIEN|      BERGEN|2006-02-15 04:34:33|
|      15|      CUBA|     OLIVIER|2006-02-15 04:34:33|
|      16|

In [14]:
from pyspark.sql import functions as F

bh_bairros_df.write \
             .mode("overwrite") \
             .option("truncate", "true") \
             .jdbc(url=jdbc_url, 
                   table="new_table",
                   properties=connection_properties)

bh_bairros_df.printSchema()

root
 |-- actor_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- last_update: timestamp (nullable = true)



## PartitionBy

In [5]:
parquet_df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Games: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Event: string (nullable = true)
 |-- Medal: string (nullable = true)



In [6]:
parquet_df.write \
          .partitionBy("City", "Year") \
          .parquet('output/partitioned', mode='overwrite')

In [12]:
spark.read.parquet("output/partitioned").printSchema()

root
 |-- ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Games: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Event: string (nullable = true)
 |-- Medal: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Year: string (nullable = true)



## Bucketing

In [25]:
parquet_df.write \
          .bucketBy(16, 'City') \
          .sortBy('City') \
          .saveAsTable("bucketed", format='parquet', mode="Overwrite")

In [26]:
spark.table('bucketed').show()

+-----+--------------------+----------+------------+------+------+---------+---+-----------+----+-----------+--------+-------------+--------------------+---------+
|   ID|                Name|       Sex|         Age|Height|Weight|     Team|NOC|      Games|Year|     Season|    City|        Sport|               Event|    Medal|
+-----+--------------------+----------+------------+------+------+---------+---+-----------+----+-----------+--------+-------------+--------------------+---------+
| 5816|"Edna Roxanne ""R...| -Campbell| -Andersen)"|     F|    24|      160| NA|     Canada| CAN|1936 Summer|    1936|       Summer|              Berlin|Athletics|
|27542|"Willemijntje ""W...|    -Jager| -Schupper)"|     F|    18|      163| NA|Netherlands| NED|1936 Summer|    1936|       Summer|              Berlin| Swimming|
|27542|"Willemijntje ""W...|    -Jager| -Schupper)"|     F|    18|      163| NA|Netherlands| NED|1936 Summer|    1936|       Summer|              Berlin| Swimming|
|   52|       Pa

2021-09-29 03:13:26,474 WARN server.TransportChannelHandler: Exception in connection from /127.0.0.1:35774
java.io.IOException: Connection reset by peer
	at java.base/sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at java.base/sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at java.base/sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:276)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:233)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:223)
	at java.base/sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:356)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:253)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1133)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:350)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:148)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:714)
	at io.netty.channel.nio.NioEventLoop.processS